In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from matplotlib import gridspec
import re
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPool1D, GRU, Flatten
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.metrics import accuracy_score

In [2]:
from keras.datasets import imdb
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
d:\Extra\pyDev\.pyDev\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
d:\Extra\pyDev\.pyDev\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do

In [3]:
X_test = data[:10000]
Y_test = targets[:10000]
X_train = data[10000:]
Y_train = targets[10000:]

In [4]:
max_review_length = 500 # макс. длина текста
embedding_vecor_length = 32 # 32-мерное векторное представление
top_words = 10000 # количество слов, рассматриваемых как признаки

In [5]:
# print(X_train[0])
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
# print(X_train[0])
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [6]:
model_a = Sequential([
    Embedding(top_words, embedding_vecor_length, input_length=max_review_length),
    Conv1D(64, 3, padding='same', activation='relu'),
    MaxPool1D(2),
    Dropout(0.3),
    Conv1D(128, 3, padding='same', activation='relu'),
    MaxPool1D(2),
    Dropout(0.2),
    LSTM(64, return_sequences=True, dropout=0.2),
    LSTM(32),
    Dense(1, activation='sigmoid')])

In [7]:
model_b = Sequential([
    Embedding(top_words, embedding_vecor_length, input_length=max_review_length),
    LSTM(100, return_sequences=True, dropout=0.3),
    LSTM(50, dropout=0.3),
    Dense(1, activation='sigmoid')])

In [8]:
model_c = Sequential([
    Embedding(top_words, embedding_vecor_length, input_length=max_review_length),
    Conv1D(32, 3, padding='same', activation='relu'),
    MaxPool1D(2),
    Dropout(0.5),
    Conv1D(64, 3, padding='same', activation='relu'),
    MaxPool1D(2),
    Dropout(0.4),
    Flatten(),
    Dense(128),
    Dropout(0.5),
    Dense(1, activation='sigmoid')])

In [9]:
ans_len = 4
train_size, test_size = len(X_train) // ans_len, len(X_test) // ans_len

In [10]:
members = [model_a, model_b, model_c]
epochs = [2, 2, 3]
for i, mod in enumerate(members):
    x_train = X_train[i * train_size : (i + 1) * train_size]
    y_train = Y_train[i * train_size : (i + 1) * train_size]
    x_test = X_test[i * test_size : (i + 1) * test_size]
    y_test = Y_test[i * test_size : (i + 1) * test_size]
    mod.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    mod.fit(x_train, y_train, validation_split=0.1, epochs=epochs[i], batch_size=64)
    scores = mod.evaluate(x_test, y_test, verbose=2)
    print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/2
141/141 [==============================] - 85s 350ms/step - loss: 0.6453 - accuracy: 0.5854 - val_loss: 0.3302 - val_accuracy: 0.8580
Epoch 2/2
141/141 [==============================] - 6s 42ms/step - loss: 0.2411 - accuracy: 0.9093 - val_loss: 0.2844 - val_accuracy: 0.8880
79/79 - 1s - loss: 0.3230 - accuracy: 0.8672
Accuracy: 86.72%
Epoch 1/2
141/141 [==============================] - 40s 126ms/step - loss: 0.6451 - accuracy: 0.5873 - val_loss: 0.3971 - val_accuracy: 0.8290
Epoch 2/2
141/141 [==============================] - 17s 117ms/step - loss: 0.3202 - accuracy: 0.8769 - val_loss: 0.3666 - val_accuracy: 0.8570
79/79 - 2s - loss: 0.4051 - accuracy: 0.8508
Accuracy: 85.08%
Epoch 1/3
141/141 [==============================] - 6s 26ms/step - loss: 0.6985 - accuracy: 0.5071 - val_loss: 0.6860 - val_accuracy: 0.5720
Epoch 2/3
141/141 [==============================] - 3s 22ms/step - loss: 0.5487 - accuracy: 0.7125 - val_loss: 0.3071 - val_accuracy: 0.8780
Epoch 3/3
141/141 

In [12]:
def stacked_prediction(members, x_test, load):
    yhat = []
    for i, m in enumerate(members):
        print('+')
        if load:
            print(m.predict(x_test, verbose=0))
        yhat.append(np.round(m.predict(x_test, verbose=0)))
    yhat = np.asarray(yhat)
    yhat = [np.round(np.mean(x)) for x in zip(*yhat)]
    return np.asarray(yhat).astype('int')

yhat = stacked_prediction(members, X_test, False)
acc = accuracy_score(Y_test, yhat)
print("Accuracy: %.2f%%" % (acc*100))

In [194]:
def load_text():
    dictionary = imdb.get_word_index()
    load_x = []
    words = input()
    words = re.sub(r"[^a-zA-Z0-9']", " ", words)
    words = words.split(' ')
    valid = []
    for word in words:
        word = dictionary.get(word)
        if word in range(1, 10000):
            valid.append(word+3)
    load_x.append(valid)
    result = sequence.pad_sequences(load_x, maxlen=max_review_length)
    print(stacked_prediction(members, result, True))

load_text() 

With so many characters, the movie spends too much time on discovery and not enough on showing those powers in action.
[[0.24501102]]
[[0.3473688]]
[[0.5906282]]
[0]
